<a href="https://colab.research.google.com/github/paishowstopper/TSAI/blob/main/Capstone/END_Capstone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

import torchtext
from torchtext.legacy import data
from torchtext.legacy.data import Field, BucketIterator

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

import spacy
import numpy as np

import random
import math
import time
import pickle
import json
import re

from tokenize import tokenize
from io import BytesIO

In [ ]:
def format_solution(solution):
    tmp_solution = []
    for sentence in solution:
        if sentence == '\n' or re.match(r' +\n|\t+\n', sentence):
            continue
        else:
            trailing_spaces = re.match(r'(^.*?)[ ]+\n', sentence)
            if trailing_spaces:
                sentence = re.sub(r'(^.*?)[ ]+\n', r'\1\n', sentence)
            tmp_solution.append(sentence)
    solution = tmp_solution

    def find_indent_value(starting_spaces_, indent_scheme_):
        sum_of_indents = 0
        for i, j in enumerate(indent_scheme_):
            sum_of_indents += j
            if sum_of_indents == starting_spaces_:
                break
        total_indents_ = i
        if total_indents_ < 0:
            raise Exception

        return total_indents_

    pruned_solution = []
    indent_scheme = []
    check_indentation_flag = True
    for sentence_number,sentence in enumerate(solution):
        starting_spaces = len(re.match(r'^([ ]*).*?', sentence)[1])
        if check_indentation_flag:
            possible_indent = starting_spaces - sum(indent_scheme)
            if possible_indent > 0 or sentence_number == 0: # checking sentence number to get the first indentation which cud be 0
                indent_scheme.append(possible_indent)
            check_indentation_flag = False
        if re.match(r'.*:\n', sentence):
            check_indentation_flag = True

        total_indents = find_indent_value(starting_spaces, indent_scheme)
        indent = total_indents * '\t'
        pruned_solution.append(re.sub(r'([ ]*)(.*?)\n', indent + r'\2\n', sentence))

    return pruned_solution

def getData(path):
    question = re.compile(r'^#[ ]?[0-9]*[.]?[ ]?(.*)')
    unique_questions_with_different_solution = {}  # used to check for unique answers
    question_answer_pair = {}  # used to save solutions for unique questions
    f = open(path, "r")

    solution = []
    for i, line in enumerate(f):
        match_object = question.match(line)
        if match_object:
            if len(solution) == 0:
                prev_match = match_object[1]
                continue
            else:
                solution_combined = " ".join(solution).replace('\n', "").replace('\t', "").replace(" ", "")
                if prev_match not in unique_questions_with_different_solution:
                    unique_questions_with_different_solution[prev_match] = [solution_combined]
                    question_answer_pair[prev_match] = [solution]
                else:
                    flag = 0
                    for j in unique_questions_with_different_solution[prev_match]:
                        if j != solution_combined:
                            flag = 1
                    if flag:
                        unique_questions_with_different_solution[prev_match].append(solution_combined)
                        question_answer_pair[prev_match].append(solution)
            solution = []
            prev_match = match_object[1]
        else:
            solution.append(line)

    questions_list = []
    answers_list = []
    for i in question_answer_pair:
        for j in question_answer_pair[i]:
            questions_list.append(i)
            formatted_solution = format_solution(j)
            answers_list.append("".join(formatted_solution))

    return questions_list,answers_list


def getTokenizer(python_code):
    tokens = []
    try:
        a = list(tokenize(BytesIO(python_code.encode('utf-8')).readline))
        for i__ in a[1:-1]:
            if i__.exact_type == 3:
                if re.match(r'^f"', i__[1]):
                    string_tokens = ['f"'] + [k__ for k__ in i__[1][2:]]
                elif re.match(r"^f'", i__[1]):
                    string_tokens = ["f'"] + [k__ for k__ in i__[1][2:]]
                else:
                    string_tokens = [k__ for k__ in i__[1]]
                tokens = tokens + string_tokens
            elif i__.exact_type == 6:   # removing dedent tokens
                continue
            else:
                tokens.append(i__[1])
    except Exception:
        print("Error in tokenization")

    return tokens

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
cd 'drive/MyDrive/capstone-data'

/content/drive/MyDrive/capstone-data


In [ ]:
questions, answers = [],[]

In [ ]:
# loading train json
f = open("conala-train.json","r")
train_file = json.load(f)
f.close()

In [ ]:
for num,i in enumerate(train_file):
  if i['intent'] is not None:
    questions.append(i['intent'])
    answers.append(i['snippet'])

In [ ]:
# loading train json
f = open("conala-test.json","r")
test_file = json.load(f)
f.close()

In [ ]:
for num,i in enumerate(test_file):
  if i['intent'] is not None:
    questions.append(i['intent'])
    answers.append(i['snippet'])

In [ ]:
questions_, answers_ = getData("session8_assignment.py")

In [ ]:
questions_, answers_ = getData("english_python_data.txt")

In [ ]:
max_word_len = 301

In [ ]:
# removing examples with len more than max_word_len
# Only checking in original data because I checked in CONALA data and answers their dont exceed 301.
pruned_questions = []
pruned_answers = []
for j,i in zip(questions_,answers_):
  tokens = getTokenizer(i)
  if not len(tokens) > max_word_len:
    pruned_answers.append(i)
    pruned_questions.append(j)

answers_ = pruned_answers
questions_ = pruned_questions

Error in tokenization
Error in tokenization
Error in tokenization
Error in tokenization
Error in tokenization
Error in tokenization
Error in tokenization
Error in tokenization
Error in tokenization
Error in tokenization


In [ ]:
questions = questions + questions_
answers = answers + answers_

In [ ]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [ ]:
!python -m spacy download en

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [ ]:
spacy_en = spacy.load('en')

In [ ]:
def tokenize_en(text):
    return [tok.text for tok in spacy_en.tokenizer(text)]

In [ ]:
SRC = Field(tokenize = tokenize_en, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True, 
            batch_first = True)

TRG = Field(tokenize = getTokenizer, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = False, 
            batch_first = True)

In [ ]:
fields = [('src', SRC), ('trg', TRG)]

Examples = [data.Example.fromlist([i,j], fields) for i,j in zip(questions,answers)]
Dataset = data.Dataset(Examples, fields)

Error in tokenization
Error in tokenization
Error in tokenization
Error in tokenization
Error in tokenization
Error in tokenization
Error in tokenization
Error in tokenization
Error in tokenization
Error in tokenization


In [ ]:
train_data,valid_data = Dataset.split(split_ratio=[0.90,0.10])

In [ ]:
SRC.build_vocab(train_data, min_freq = 1)
TRG.build_vocab(train_data, min_freq = 1)

In [ ]:
# Dumps dicts
with open("SRC_stio_local","wb") as f:
  pickle.dump(SRC.vocab.stoi,f)
with open("TRG_itos_local","wb") as f:
  pickle.dump(TRG.vocab.itos,f)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
BATCH_SIZE = 32

train_iterator, valid_iterator = BucketIterator.splits(
    (train_data, valid_data), 
     batch_size = BATCH_SIZE,
     sort_key = lambda x: len(x.trg),
     device = device)

In [ ]:
class PositionalEncodingComponent(nn.Module):
  '''
  Class to encode positional information to tokens.
  '''
  def __init__(self,hid_dim,device,dropout=0.2,max_len=5000):
    super().__init__()

    assert hid_dim%2==0 # If not, it will result error in allocation to positional_encodings[:,1::2] later

    self.dropout = nn.Dropout(dropout)

    self.positional_encodings = torch.zeros(max_len,hid_dim)

    pos = torch.arange(0,max_len).unsqueeze(1) # pos : [max_len,1]
    div_term  = torch.exp(-torch.arange(0,hid_dim,2)*math.log(10000.0)/hid_dim) # Calculating value of 1/(10000^(2i/hid_dim)) in log space and then exponentiating it
    # div_term: [hid_dim//2]

    self.positional_encodings[:,0::2] = torch.sin(pos*div_term) # pos*div_term [max_len,hid_dim//2]
    self.positional_encodings[:,1::2] = torch.cos(pos*div_term) 

    self.positional_encodings = self.positional_encodings.unsqueeze(0) # To account for batch_size in inputs

    self.device = device

  def forward(self,x):
    x = x + self.positional_encodings[:,:x.size(1)].detach().to(self.device)
    return self.dropout(x)

In [ ]:
class PositionwiseFeedforwardLayer(nn.Module):
  def __init__(self, hid_dim, pf_dim, dropout):

    super().__init__()

    self.fc_1 = nn.Linear(hid_dim, pf_dim)
    self.fc_2 = nn.Linear(pf_dim, hid_dim)

    self.dropout = nn.Dropout(dropout)

  def forward(self, x):

    x = self.dropout(torch.relu(self.fc_1(x)))
    x = self.fc_2(x)

    return x

In [ ]:
class MultiHeadAttentionLayer(nn.Module):
  def __init__(self, hid_dim, n_heads, dropout, device):

    super().__init__()

    assert hid_dim % n_heads == 0

    self.hid_dim = hid_dim
    self.n_heads = n_heads
    self.head_dim = hid_dim // n_heads

    self.fc_q = nn.Linear(hid_dim, hid_dim)
    self.fc_k = nn.Linear(hid_dim, hid_dim)
    self.fc_v = nn.Linear(hid_dim, hid_dim)

    self.fc_o = nn.Linear(hid_dim, hid_dim)

    self.dropout = nn.Dropout(dropout)

    self.scale = torch.sqrt(torch.FloatTensor([self.head_dim])).to(device)

  def forward(self, query, key, value, mask = None):

    batch_size = query.shape[0]

    Q = self.fc_q(query)
    K = self.fc_k(key)
    V = self.fc_v(value)

    Q = Q.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
    K = K.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
    V = V.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)

    energy = torch.matmul(Q, K.permute(0, 1, 3, 2)) / self.scale

    if mask is not None:
      energy = energy.masked_fill(mask == 0, -1e10)

    attention = torch.softmax(energy, dim = -1)

    x = torch.matmul(self.dropout(attention), V)
    x = x.permute(0, 2, 1, 3).contiguous()
    x = x.view(batch_size, -1, self.hid_dim)
    x = self.fc_o(x)

    return x, attention

In [ ]:
class EncoderLayer(nn.Module):
  def __init__(self,
               hid_dim,
               n_heads,
               pf_dim,
               dropout,
               device):

    super().__init__()

    self.self_attn_layer_norm = nn.LayerNorm(hid_dim)
    self.ff_layer_norm = nn.LayerNorm(hid_dim)
    self.self_attention = MultiHeadAttentionLayer(hid_dim, n_heads, dropout, device)
    self.positionwise_feedforward = PositionwiseFeedforwardLayer(hid_dim,
                                                                 pf_dim,
                                                                 dropout)

    self.dropout = nn.Dropout(dropout)
  
  def forward(self, src, src_mask):

    _src, _ = self.self_attention(src, src, src, src_mask)
    src = self.self_attn_layer_norm(src + self.dropout(_src))

    _src = self.positionwise_feedforward(src)
    src = self.ff_layer_norm(src + self.dropout(_src))

    return src

In [ ]:
class DecoderLayer(nn.Module):
  def __init__(self,
               hid_dim,
               n_heads,
               pf_dim,
               dropout,
               device):

    super().__init__()

    self.self_attn_layer_norm = nn.LayerNorm(hid_dim)
    self.enc_attn_layer_norm = nn.LayerNorm(hid_dim)
    self.ff_layer_norm = nn.LayerNorm(hid_dim)
    self.self_attention = MultiHeadAttentionLayer(hid_dim, n_heads, dropout, device)
    self.encoder_attention = MultiHeadAttentionLayer(hid_dim, n_heads, dropout, device)
    self.positionwise_feedforward = PositionwiseFeedforwardLayer(hid_dim,
                                                                 pf_dim,
                                                                 dropout)

    self.dropout = nn.Dropout(dropout)
  
  def forward(self, trg, enc_src, trg_mask, src_mask):

    _trg, _ = self.self_attention(trg, trg, trg, trg_mask)
    trg = self.self_attn_layer_norm(trg + self.dropout(_trg))

    _trg, attention = self.encoder_attention(trg, enc_src, enc_src, src_mask)
    trg = self.enc_attn_layer_norm(trg + self.dropout(_trg))

    _trg = self.positionwise_feedforward(trg)
    trg = self.ff_layer_norm(trg + self.dropout(_trg))

    return trg, attention

In [ ]:
# class Encoder(nn.Module):
#   def __init__(self,
#                input_dim,
#                hid_dim,
#                n_layers,
#                n_heads,
#                pf_dim,
#                dropout,
#                device,
#                max_length=100):

#     super().__init__()

#     self.device = device

#     self.tok_embedding = nn.Embedding(input_dim, hid_dim)
#     self.pos_embedding = nn.Embedding(max_length, hid_dim)

#     self.layers = nn.ModuleList([EncoderLayer(hid_dim,
#                                               n_heads,
#                                               pf_dim,
#                                               dropout,
#                                               device)
#                                 for _ in range(n_layers)])
    
#     self.dropout = nn.Dropout(dropout)
#     self.scale = torch.sqrt(torch.FloatTensor([hid_dim])).to(device)

#   def forward(self, src, src_mask):

#     batch_size = src.shape[0]
#     src_len = src.shape[1]

#     pos = torch.arange(0, src_len).unsqueeze(0).repeat(batch_size, 1).to(self.device)

#     src = self.dropout((self.tok_embedding(src) * self.scale) + self.pos_embedding(pos))

#     for layer in self.layers:
#       src = layer(src, src_mask)

#     return src  

In [ ]:
class Encoder(nn.Module):
  def __init__(self,input_dim,hid_dim,n_layers,n_heads,pf_dim,dropout,device,max_length = 5000):
    super().__init__()
    self.device = device

    self.tok_embedding = nn.Embedding(input_dim,hid_dim)
    self.pos_embedding = PositionalEncodingComponent(hid_dim,device,dropout,max_length)

    self.layers = nn.ModuleList([EncoderLayer(hid_dim,n_heads,pf_dim,dropout,device) for _ in range(n_layers)])

    self.dropout = nn.Dropout(dropout)

    self.scale = torch.sqrt(torch.FloatTensor([hid_dim])).to(device)

  def forward(self,src,src_mask):

    batch_size = src.shape[0]
    src_len = src.shape[1]

    tok_embeddings = self.tok_embedding(src)*self.scale
    src  = self.pos_embedding(tok_embeddings)

    for layer in self.layers:
      src = layer(src,src_mask)

    return src

In [ ]:
# class Decoder(nn.Module):
#   def __init__(self,
#                output_dim,
#                hid_dim,
#                n_layers,
#                n_heads,
#                pf_dim,
#                dropout,
#                device,
#                max_length=100):

#     super().__init__()

#     self.device = device

#     self.tok_embedding = nn.Embedding(output_dim, hid_dim)
#     self.pos_embedding = nn.Embedding(max_length, hid_dim)

#     self.layers = nn.ModuleList([DecoderLayer(hid_dim,
#                                               n_heads,
#                                               pf_dim,
#                                               dropout,
#                                               device)
#                                 for _ in range(n_layers)])
    
#     self.fc_out = nn.Linear(hid_dim, output_dim)
#     self.dropout = nn.Dropout(dropout)
#     self.scale = torch.sqrt(torch.FloatTensor([hid_dim])).to(device)

#   def forward(self, trg, enc_src, trg_mask, src_mask):

#     batch_size = trg.shape[0]
#     trg_len = trg.shape[1]

#     pos = torch.arange(0, trg_len).unsqueeze(0).repeat(batch_size, 1).to(self.device)

#     trg = self.dropout((self.tok_embedding(trg) * self.scale) + self.pos_embedding(pos))

#     for layer in self.layers:
#       trg, attention = layer(trg, enc_src, trg_mask, src_mask)

#     output = self.fc_out(trg)

#     return output, attention 

In [ ]:
class Decoder(nn.Module):
  def __init__(self,output_dim,hid_dim,n_layers,n_heads,pf_dim,dropout,device,max_length= 5000):
    super().__init__()

    self.device = device

    self.tok_embedding = nn.Embedding(output_dim,hid_dim)
    self.pos_embedding = PositionalEncodingComponent(hid_dim,device,dropout,max_length)

    self.layers = nn.ModuleList([DecoderLayer(hid_dim,n_heads,pf_dim,dropout,device) for _ in range(n_layers)])

    self.fc_out = nn.Linear(hid_dim,output_dim)
    self.dropout = nn.Dropout(dropout)
    self.scale = torch.sqrt(torch.FloatTensor([hid_dim])).to(device)

  def forward(self, trg, enc_src,trg_mask,src_mask):
    
    batch_size = trg.shape[0]
    trg_len = trg.shape[1]

    tok_embeddings = self.tok_embedding(trg)*self.scale

    trg = self.pos_embedding(tok_embeddings)

    for layer in self.layers:
      trg, encoder_attention = layer(trg,enc_src,trg_mask,src_mask)
    
    output = self.fc_out(trg)
    
    return output, encoder_attention

In [ ]:
class Seq2Seq(nn.Module):
  def __init__(self,
               encoder,
               decoder,
               src_pad_idx,
               trg_pad_idx,
               device):

    super().__init__()

    self.encoder = encoder
    self.decoder = decoder
    self.src_pad_idx = src_pad_idx
    self.trg_pad_idx = trg_pad_idx
    self.device = device

  def make_src_mask(self, src):

    src_mask = (src != self.src_pad_idx).unsqueeze(1).unsqueeze(2)
    return src_mask

  def make_trg_mask(self, trg):

    trg_pad_mask = (trg != self.trg_pad_idx).unsqueeze(1).unsqueeze(2)

    trg_len = trg.shape[1]

    trg_sub_mask = torch.tril(torch.ones((trg_len, trg_len), device = self.device)).bool()

    trg_mask = trg_pad_mask & trg_sub_mask

    return trg_mask

  def forward(self, src, trg):

    src_mask = self.make_src_mask(src)
    trg_mask = self.make_trg_mask(trg)

    enc_src = self.encoder(src, src_mask)

    output, attention = self.decoder(trg, enc_src, trg_mask, src_mask)

    return output, attention

In [ ]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
HID_DIM = 256
ENC_LAYERS = 3
DEC_LAYERS = 3
ENC_HEADS = 8
DEC_HEADS = 8
ENC_PF_DIM = 512
DEC_PF_DIM = 512
ENC_DROPOUT = 0.1
DEC_DROPOUT = 0.1

enc = Encoder(INPUT_DIM, 
              HID_DIM, 
              ENC_LAYERS, 
              ENC_HEADS, 
              ENC_PF_DIM, 
              ENC_DROPOUT, 
              device)

dec = Decoder(OUTPUT_DIM, 
              HID_DIM, 
              DEC_LAYERS, 
              DEC_HEADS, 
              DEC_PF_DIM, 
              DEC_DROPOUT, 
              device)

SRC_PAD_IDX = SRC.vocab.stoi[SRC.pad_token]
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]

model = Seq2Seq(enc, dec, SRC_PAD_IDX, TRG_PAD_IDX, device).to(device)

In [ ]:
def initialize_weights(m):
    if hasattr(m, 'weight') and m.weight.dim() > 1:
        nn.init.xavier_uniform_(m.weight.data)

model.apply(initialize_weights);

In [ ]:
# load python embedding weights
pretrained_embeddings = torch.load('python_embedding_weigts.pt')

In [ ]:
# Loading pretrained embeddings
with open("TRG_stio","rb") as f:
  stoi_weights = pickle.load(f)

with torch.no_grad():
  indexes = []
  for i,j in enumerate(TRG.vocab.stoi):
    if j in stoi_weights:
      model.decoder.tok_embedding.weight[TRG.vocab.stoi[j]] = pretrained_embeddings.weight[stoi_weights[j]]
    else:
      model.decoder.tok_embedding.weight[TRG.vocab.stoi[j]] = pretrained_embeddings.weight[stoi_weights['<unk>']]

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 7,140,960 trainable parameters


In [ ]:
LEARNING_RATE = 0.0001
optimizer = torch.optim.Adam(model.parameters(), lr = LEARNING_RATE)
criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)
# criterion = nn.KLDivLoss()

In [ ]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.src
        trg = batch.trg
        
        optimizer.zero_grad()
        
        output, _ = model(src, trg[:,:-1])
        output_dim = output.shape[-1]
            
        output = output.contiguous().view(-1, output_dim)
        trg = trg[:,1:].contiguous().view(-1)
                
        loss = criterion(output, trg)
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.src
            trg = batch.trg

            output, _ = model(src, trg[:,:-1])
            output_dim = output.shape[-1]
            
            output = output.contiguous().view(-1, output_dim)
            trg = trg[:,1:].contiguous().view(-1)
            
            loss = criterion(output, trg)
            epoch_loss += loss.item()

    return epoch_loss / len(iterator)

In [ ]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
N_EPOCHS = 50
CLIP = 1

best_valid_loss = float('inf')
best_train_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'conala_plus_original_data.pt')
    
    if train_loss < best_train_loss:
        best_train_loss = train_loss
        torch.save(model.state_dict(), 'best_train_loss.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 0m 18s
	Train Loss: 5.567 | Train PPL: 261.598
	 Val. Loss: 3.930 |  Val. PPL:  50.903
Epoch: 02 | Time: 0m 17s
	Train Loss: 3.636 | Train PPL:  37.933
	 Val. Loss: 3.233 |  Val. PPL:  25.349
Epoch: 03 | Time: 0m 17s
	Train Loss: 3.190 | Train PPL:  24.300
	 Val. Loss: 2.962 |  Val. PPL:  19.341
Epoch: 04 | Time: 0m 17s
	Train Loss: 2.962 | Train PPL:  19.345
	 Val. Loss: 2.813 |  Val. PPL:  16.659
Epoch: 05 | Time: 0m 17s
	Train Loss: 2.801 | Train PPL:  16.465
	 Val. Loss: 2.676 |  Val. PPL:  14.531
Epoch: 06 | Time: 0m 17s
	Train Loss: 2.672 | Train PPL:  14.470
	 Val. Loss: 2.587 |  Val. PPL:  13.290
Epoch: 07 | Time: 0m 17s
	Train Loss: 2.567 | Train PPL:  13.024
	 Val. Loss: 2.504 |  Val. PPL:  12.236
Epoch: 08 | Time: 0m 17s
	Train Loss: 2.474 | Train PPL:  11.865
	 Val. Loss: 2.444 |  Val. PPL:  11.515
Epoch: 09 | Time: 0m 17s
	Train Loss: 2.395 | Train PPL:  10.964
	 Val. Loss: 2.394 |  Val. PPL:  10.962
Epoch: 10 | Time: 0m 17s
	Train Loss: 2.321 | Train PPL

In [ ]:
import pickle

with open("SRC_stio_local","rb") as f:
  stoi = pickle.load(f)
with open("TRG_itos_local","rb") as f:
  itos = pickle.load(f)

# Load model
# trained_model = 'conala_plus_original_data.pt'
trained_model = 'best_train_loss.pt'
model.load_state_dict(torch.load(trained_model));
model.eval()

Seq2Seq(
  (encoder): Encoder(
    (tok_embedding): Embedding(3024, 256)
    (pos_embedding): PositionalEncodingComponent(
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (layers): ModuleList(
      (0): EncoderLayer(
        (self_attn_layer_norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (ff_layer_norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (self_attention): MultiHeadAttentionLayer(
          (fc_q): Linear(in_features=256, out_features=256, bias=True)
          (fc_k): Linear(in_features=256, out_features=256, bias=True)
          (fc_v): Linear(in_features=256, out_features=256, bias=True)
          (fc_o): Linear(in_features=256, out_features=256, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (positionwise_feedforward): PositionwiseFeedforwardLayer(
          (fc_1): Linear(in_features=256, out_features=512, bias=True)
          (fc_2): Linear(in_features=512, out_features=256, bias=True)
    

In [ ]:
import spacy
spacy_en = spacy.load('en')

def encode_inputs(input,vocab):

  tokenized_input_ = [tok.text.lower() for tok in spacy_en.tokenizer(input)]
  tokenized_input = ['<sos>'] + tokenized_input_ +['<eos>']

  numericalized_input = [vocab[i] for i in tokenized_input]

  tensor_input = torch.LongTensor([numericalized_input])
  
  return tensor_input,tokenized_input_

def decode_outputs(output,vocab):
  # output: [1,1,hid_dim]
  predicted_token = output.argmax(-1)
  return vocab[predicted_token.item()], predicted_token

In [ ]:
def display_attention(sentence, translation, attention, n_heads = 8, n_rows = 4, n_cols = 2):
    
    assert n_rows * n_cols == n_heads
    
    fig = plt.figure(figsize=(15,25))
    
    for i in range(n_heads):
        
        ax = fig.add_subplot(n_rows, n_cols, i+1)
        
        _attention = attention.squeeze(0)[i].cpu().detach().numpy()

        cax = ax.matshow(_attention, cmap='bone')

        ax.tick_params(labelsize=12)
        ax.set_xticklabels(['']+['<sos>']+[t.lower() for t in sentence]+['<eos>'], 
                           rotation=45)
        ax.set_yticklabels(['']+translation)

        ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
        ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.show()

In [ ]:
def print_decoder_output(decoder_outputs):
  # print(decoder_outputs)
  combined_output = " ".join(decoder_outputs)
  pruned_output = re.sub(r'\n |\n  ',r'\n',combined_output)
  # pruned_output = re.sub(" . ",".",pruned_output)
  # pruned_output = combined_output
  print(pruned_output)


In [ ]:
 print(" Enter q or quit to exit.")

answer_max_len = 200

while(True):

  input_ = input("Enter text:")

  if input_=='q' or input_=='quit':
    break

  src,tokenized_input_ = encode_inputs(input_,stoi)
  src = src.to(device)
  # src_mask = torch.ones([1,1,1,src.shape[-1]]).to(device)
  src_mask = model.make_src_mask(src)

  with torch.no_grad():
    enc_src = model.encoder(src,src_mask)
  
  trg = '<sos>'
  trg_indexes = [stoi[trg]]
  # trg_mask = torch.ones([1,1,1,1]).to(device)

  decoder_outputs = []
  for i in range(answer_max_len):
    trg_tensor = torch.LongTensor(trg_indexes).unsqueeze(0).to(device)
    trg_mask = model.make_trg_mask(trg_tensor)
    
    with torch.no_grad():
      decoder_output,encoder_decoder_attention = model.decoder(trg_tensor,enc_src,trg_mask,src_mask)

    pred_token = decoder_output.argmax(2)[:,-1].item()

    if pred_token == TRG.vocab.stoi[TRG.eos_token]:
      break
    decoder_outputs.append(itos[pred_token])
    trg_indexes.append(pred_token)


  print_decoder_output(decoder_outputs)
  # display_attention(tokenized_input_,decoder_outputs,encoder_decoder_attention,DEC_HEADS)

 Enter q or quit to exit.
Enter text:program to calculate simple interest
def simple_interest ( p , r ) : 
	 si = ( p * t * r ) / 100 
return si 
Enter text:function to sum odd elements of list
sum ( sum ( i for i in range ( len ( l ) ) ) 
Enter text:reverse a string
' ' . join ( reversed ( reversed ( reversed ( reversed ( reversed ( reversed ( reversed ( s ) ) ) ) ) ) ) 
